# Lab 05: Pytorch

## Multiclass Classification

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
Y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.15, random_state=42)

In [ ]:
class Data(Dataset):
    def __init__(self, X, y):
        self.x = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
        self.len = self.x.shape[0]
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [ ]:
train_dataset = Data(x_train, y_train)
trainloader = DataLoader(dataset=train_dataset,batch_size=64)
val_dataset = Data(x_val, y_val)
val_loader = DataLoader(dataset=val_dataset,batch_size=64)
test_dataset = Data(x_train, y_train)
test_loader = DataLoader(dataset=test_dataset,batch_size=64)

In [ ]:
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)

    def forward(self,x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

In [ ]:
input_dim = 4
hidden_dim = 25
output_dim = 3
model = Net(input_dim,hidden_dim,output_dim)

**Requirement:** Write script to Train and Inference

* In Train script must log loss, Accuracy, Precision, Recall, F1.

  *Hint:*

  https://torchmetrics.readthedocs.io/en/v0.10.2/classification/precision_recall.html

  https://lightning.ai/docs/torchmetrics/stable/classification/f1_score.html

  https://lightning.ai/docs/torchmetrics/stable/classification/accuracy.html

* View `classification_report` by `sklearn` in Inference script.

## Assignment

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

### Dataset

In [ ]:
# load dataset
from sklearn.datasets import load_iris
iris = load_iris()
# features, targets split
X = iris.data
Y = iris.target

In [ ]:
X.shape

(150, 4)

In [ ]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

**Bộ dữ liệu Iris**

* Bộ dữ liệu gồm 150 mẫu, chia thành 3 lớp, mỗi lớp đại diện cho một loài hoa Iris: **Iris Setosa**, **Iris Versicolor**, và **Iris Virginica**.

* Mỗi mẫu có 4 đặc trưng đo bằng cm:
  * Chiều dài đài hoa (`Sepal Length`)
  * Chiều rộng đài hoa (`Sepal Width`)
  * Chiều dài cánh hoa (`Petal Length`)
  * Chiều rộng cánh hoa (`Petal Width`)

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.15, random_state=42)

In [ ]:
# Data object
class Data(Dataset):
    def __init__(self, X, y):
        self.x = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
        self.len = self.x.shape[0]
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [ ]:
# DataLoader
train_dataset = Data(x_train, y_train)
trainloader = DataLoader(dataset=train_dataset,batch_size=64)
val_dataset = Data(x_val, y_val)
val_loader = DataLoader(dataset=val_dataset,batch_size=64)
test_dataset = Data(x_train, y_train)
test_loader = DataLoader(dataset=test_dataset,batch_size=64)

### Model

1. Design Model

In [ ]:
# model = nn.Net(input_dim, output_dim)
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        # define different layers
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)

    def forward(self,x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

In [ ]:
input_dim = 4
hidden_dim = 25
output_dim = 3
model = Net(input_dim,hidden_dim,output_dim)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Set the device
model = model.to(device)  # Move the model to the device

2. Define loss and optimizer

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

3. Train Model

In [ ]:
!pip install torchmetrics

In [ ]:
from torchmetrics import Accuracy, Precision, Recall, F1Score

# define metrics
accuracy_metric = Accuracy(task="multiclass", num_classes=output_dim, average='weighted')
precision_metric = Precision(task="multiclass", num_classes=output_dim, average='weighted')
recall_metric = Recall(task="multiclass", num_classes=output_dim, average='weighted')
f1_metric = F1Score(task="multiclass", num_classes=output_dim, average='weighted')

In [ ]:
from tqdm import tqdm

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    # Training pharse
    model.train()
    total_loss = 0.0
    all_preds = []
    all_labels = []

    for x_batch, y_batch in tqdm(trainloader, disable=True):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad() # zero gradients after updating
        pred = model(x_batch)

        loss_value = loss(pred, y_batch) # loss
        loss_value.backward() # calculate gradients
        optimizer.step() # update weights

        total_loss += loss_value.item()
        all_preds.extend(pred.cpu().detach().numpy())
        all_labels.extend(y_batch.cpu().detach().numpy())

    # Calculate training accuracy and F1 score
    all_preds = torch.as_tensor(all_preds)
    all_labels = torch.as_tensor(all_labels)
    train_accuracy = accuracy_metric(all_preds, all_labels)
    train_precision = precision_metric(all_preds, all_labels)
    train_recall = recall_metric(all_preds, all_labels)
    train_f1 = f1_metric(all_preds, all_labels)
    avg_train_loss = total_loss / len(trainloader)

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}] - '
              f'Train: (Loss: {avg_train_loss:.4f}, '
              f'Accuracy: {train_accuracy:.4f}, '
              f'Precision: {train_precision:.4f}, '
              f'Recall: {train_recall:.4f}, '
              f'F1 Score: {train_f1:.4f})')

    # Validation phase
    model.eval()
    val_total_loss = 0
    val_all_preds = []
    val_all_labels = []

    with torch.no_grad():
        for x_batch, y_batch in tqdm(val_loader, disable=True):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            pred = model(x_batch)
            val_loss = loss(pred, y_batch)

            val_total_loss += val_loss.item()
            val_all_preds.extend(pred.cpu().detach().numpy())
            val_all_labels.extend(y_batch.cpu().detach().numpy())

    # Calculate validation accuracy and F1 score
    all_preds = torch.as_tensor(val_all_preds)
    all_labels = torch.as_tensor(val_all_labels)
    val_accuracy = accuracy_metric(all_preds, all_labels)
    val_precision = precision_metric(all_preds, all_labels)
    val_recall = recall_metric(all_preds, all_labels)
    val_f1 = f1_metric(all_preds, all_labels)
    avg_val_loss = val_total_loss / len(val_loader)

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}] - '
              f'Validation: (Loss: {avg_val_loss:.4f}, '
              f'Accuracy: {val_accuracy:.4f}, '
              f'Precision: {val_precision:.4f}, '
              f'Recall: {val_recall:.4f}, '
              f'F1 Score: {val_f1:.4f})\n')

Epoch [1/1000] - Train: (Loss: 1.1823, Accuracy: 0.1600, Precision: 0.1191, Recall: 0.1600, F1 Score: 0.1366)
Epoch [1/1000] - Validation: (Loss: 1.1705, Accuracy: 0.1429, Precision: 0.0779, Recall: 0.1429, F1 Score: 0.1008)

Epoch [101/1000] - Train: (Loss: 0.7879, Accuracy: 0.8500, Precision: 0.8959, Recall: 0.8500, F1 Score: 0.8431)
Epoch [101/1000] - Validation: (Loss: 0.7561, Accuracy: 0.7857, Precision: 0.8696, Recall: 0.7857, F1 Score: 0.7475)

Epoch [201/1000] - Train: (Loss: 0.4806, Accuracy: 0.9600, Precision: 0.9642, Recall: 0.9600, F1 Score: 0.9599)
Epoch [201/1000] - Validation: (Loss: 0.4330, Accuracy: 0.9762, Precision: 0.9778, Recall: 0.9762, F1 Score: 0.9761)

Epoch [301/1000] - Train: (Loss: 0.3254, Accuracy: 0.9700, Precision: 0.9724, Recall: 0.9700, F1 Score: 0.9700)
Epoch [301/1000] - Validation: (Loss: 0.2965, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000)

Epoch [401/1000] - Train: (Loss: 0.2206, Accuracy: 0.9700, Precision: 0.9724, Recall

4. Inference

In [ ]:
from sklearn.metrics import classification_report

# Assuming you have your test dataloader and model set up
all_labels = []
all_preds = []

# Evaluate the model on the test DataLoader
model.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Disable gradient calculation to save storage
    for x, y in test_loader:  # Replace with your test dataloader
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)  # Your model's forward pass
        _, preds = torch.max(outputs, 1)  # Get the predicted class

        all_labels.extend(y.cpu().numpy())  # Collect true labels
        all_preds.extend(preds.cpu().numpy())  # Collect predictions

# Convert lists to numpy arrays
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# Generate and print the classification report
report = classification_report(all_labels, all_preds, target_names=iris.target_names)  # Adjust class names as needed
print(report)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        31
  versicolor       1.00      0.94      0.97        35
   virginica       0.94      1.00      0.97        34

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100

